**Merhaba!**

Bu projede, gerçek bir çağrı merkezinin anonimleştirilmiş çağrı detay kayıtları (CDR) üzerinde RFM (Recency, Frequency, Monetary) analizi gerçekleştirilmiştir. Amaç, müşteri etkileşimlerini daha iyi anlamak ve değerli müşterileri tespit etmektir.

**Veri Seti**

Veri seti, müşteri çağrılarına dair detaylı bilgiler içermektedir. 'Arayan' ve 'Aranan' sütunları özel bilgiler içerdiği için **SHA-256** hash fonksiyonu kullanılarak anonimleştirilmiştir. Aynı zamanda 'Müşteri Temsilcisi' sütununu anonimleştirmek için de Faker kütüphanesi kullanılmış ve temsilciler için random isimler üretilmiştir. Bununla birlikte tarih verileri de farklı bir zamana ait olsalar da değiştirmiş, böylelikle veri setinin tam olarak anonimleştirilmesi sağlanmıştır.

RFM modeli, müşterilerin son çağrı tarihine (Recency), çağrı sıklığına (Frequency) ve toplam görüşme süresine (Monetary) göre segmentlere ayrılmasına dayanmaktadır. Burada standart yönelimin aksine, 'Görüşme Süresi' monetery değeri olarak kullanılmış ancak en düşük değer en yüksek puana sahip olacak şekilde bir puanlama sistemi uygulanmıştır.

Çağrı Merkezi yönetiminde yöneticilerin ve uzmanların değerli içgörülere sahip olmasını sağlayabilir. Sık arayan müşteri profilinin yanı sıra, uzun süreli görüşmeler yapan müşterilerin tespiti, müşteri temsilcisi ya da süreç kaynaklı iyileştirmeler için yol gösterici olabilir.

Haydi başlayalım.

In [17]:
import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

In [18]:
df = pd.read_csv("/kaggle/input/call-center-cdr/CDR_Anonymized.csv")
df.head()

,Çağrı Tarihi,Arayan,Aranan,Süre,Bekleme Süresi,Görüşme Süresi,Durum,Müşteri Temsilcisi
0,2023-11-01 00:00:00,d872ad70dea4d64ea4d868a6e6feef4ee4c635d4abccdf...,cde3d0ad71ed3bf2c37b8ef76b75f2df367a88923803d2...,00:07:08,00:00:02,150.0,Arayan Kapattı,Peter Cain
1,2023-11-01 00:00:00,2f302204c451cd9a0d9f1b23ad9733b3dd71ea3f74a892...,cde3d0ad71ed3bf2c37b8ef76b75f2df367a88923803d2...,00:00:00,NaN,0.0,Cevaplanan,Brian Gross
2,2023-11-01 00:01:00,00061a5638d32c4c876dc5117c268692dd323100f8eac4...,fb0669c6de70882cca5960d58974ccb311803f887e688c...,00:01:10,NaN,70.0,Cevaplanan,NaN
3,2023-11-01 00:04:00,53df623d158e237b41e476e9224600c6e51e32a5c93b2e...,cde3d0ad71ed3bf2c37b8ef76b75f2df367a88923803d2...,00:00:49,00:00:01,36.0,Arayan Kapattı,Brian Gross
4,2023-11-01 00:17:00,2812be1db037be1238f20e12a4908299e92678c276b2c1...,cde3d0ad71ed3bf2c37b8ef76b75f2df367a88923803d2...,00:01:40,00:00:01,87.0,Arayan Kapattı,Brian Gross


In [19]:
#Veri setinin içerisindeki sütunları gözlemleyelim.
df.columns

Index(['Çağrı Tarihi', 'Arayan', 'Aranan', 'Süre', 'Bekleme Süresi', 'Görüşme Süresi', 'Durum', 'Müşteri Temsilcisi'], dtype='object')

In [20]:
df["Çağrı Tarihi"].max()

today_date = dt.datetime(2023, 11, 7)
df['Görüşme Süresi'] = pd.to_timedelta(df['Görüşme Süresi']).dt.total_seconds()
df['Çağrı Tarihi'] = pd.to_datetime(df['Çağrı Tarihi'])

In [21]:
#RFM Tablosunu oluşturmaya başlayalım.

rfm_table = df.groupby('Arayan').agg({
    'Çağrı Tarihi': lambda x: (today_date - x.max()).days,
    'Arayan': 'count',
    'Görüşme Süresi': 'sum'})

In [22]:
#Kolonları, Recency, Frequency ve Monetary gibi global isimlendirmelerle değiştiriyoruz.
rfm_table.rename(columns={'Çağrı Tarihi': 'Recency',
                          'Arayan': 'Frequency',
                          'Görüşme Süresi': 'Monetary'}, inplace=True)

In [23]:
# RFM skorlarını hesaplama (Monetary'i unutmuyoruz. Çağrı ne kadar uzun, o kadar kötü)

rfm_table['Recency_Score'] = pd.qcut(rfm_table['Recency'], 5, labels=[5, 4, 3, 2, 1])
rfm_table['Frequency_Score'] = pd.qcut(rfm_table['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
rfm_table['Monetary_Score'] = pd.qcut(rfm_table['Monetary'], 5, labels=[5, 4, 3, 2, 1])

In [24]:
rfm_table['RFM_Score'] = rfm_table['Recency_Score'].astype(str) + rfm_table['Frequency_Score'].astype(str)
rfm_table.reset_index(inplace=True)

In [25]:
#RFM Segmentleri ve Analiz
#Dilerseniz burada da global etiketlerin dışına çıkabilirsiniz. Örneğin "sık arayan", "çözüm odaklı", "ilgilenilmesi gereken" 
#gibi sektöre özel ya da özgü isimlendirmeler kullanılabilir. Biz şimdilik buradan devam edelim. 


#seg_map = {
    #r'[1-2][1-2]': 'hibernating',
    #r'[1-2][3-4]': 'at_Risk',
    #r'[1-2]5': 'cant_loose',
    #r'3[1-2]': 'about_to_sleep',
    #r'33': 'need_attention',
    #r'[3-4][4-5]': 'loyal_customers',
    #r'41': 'promising',
    #r'51': 'new_customers',
    #r'[4-5][2-3]': 'potential_loyalists',
    #r'5[4-5]': 'champions'
#}


Dilerseniz yukarıdaki global isimlerdirmeleri kullanabileceğinizi söylemiştik.
aşağıda, RFM segmentasyonlarının çağrı merkezi bağlamında yeniden değerlendirilerek oluşturulmuş bir örneğini görebilirsiniz.

Bu örnekte, sık arayan müşterilerimiz, eğer aradıkları çağrı merkezi operasyonu talep, şikayet ya da destek hizmeti veriyorsa sorunları olan ya da sorunları çözülemeyen aboneler diyebiliriz. Burada "konuşma süresi" de bir kısmı müşteri temsilcisiden kaynaklı olsa da sorunu çözülmemiş ya da destek alamamış müşterilerimizi tarif eder.

* **hibernating** yerine **low_interaction**: Düşük etkileşimde bulunan müşteriler.
* **at_Risk** yerine **escalating_concerns**: Sıkça arayan ve sorunları artan müşteriler.
* **cant_loose** yerine **critical_attention**: Önemli ve dikkat gerektiren durumları olan müşteriler.
* **about_to_sleep** yerine **decreasing_engagement**: Azalan etkileşim ve ilgi gösteren müşteriler.
* **need_attention** yerine **steady_inquiries**: Sürekli soru sorma eğiliminde olan müşteriler.
* **loyal_customers** yerine **repeated_interactions**: Sıkça arayarak etkileşimde bulunan müşteriler.
* **promising** yerine **emerging_concerns**: Potansiyel sorunları olan ve gözlemlenen yeni müşteriler.
* **new_customers** yerine **initial_contacts**: İlk kez temas kuran yeni müşteriler.
* **potential_loyalists** yerine **increasing_contacts**: Sıkça arama yapmaya başlayan ve artan etkileşim gösteren müşteriler.
* **champions** yerine **persistent_issues**: Sürekli sorunlar yaşayan ve sıkça ulaşan müşteriler.

In [26]:
seg_map = {
    r'([1-2][1-2])': 'low_interaction',       # Düşük etkileşimli müşteriler
    r'([1-2][3-4])': 'escalating_concerns',   # Artan sorunları olan müşteriler
    r'([1-2]5)': 'critical_attention',        # Dikkat gerektiren kritik durumları olan müşteriler
    r'(3[1-2])': 'decreasing_engagement',     # Azalan etkileşim gösteren müşteriler
    r'(33)': 'steady_inquiries',              # Sürekli soru sorma eğiliminde olan müşteriler
    r'([3-4][4-5])': 'repeated_interactions', # Sıkça etkileşimde bulunan müşteriler
    r'(41)': 'emerging_concerns',             # Potansiyel sorunları olan yeni müşteriler
    r'(51)': 'initial_contacts',              # İlk kez temas kuran yeni müşteriler
    r'([4-5][2-3])': 'increasing_contacts',   # Artan etkileşim gösteren müşteriler
    r'(5[4-5])': 'persistent_issues'          # Sürekli sorun yaşayan ve sıkça ulaşan müşteriler
}


In [27]:
rfm_table['segment'] = rfm_table['RFM_Score'].replace(seg_map, regex=True)
RFM = rfm_table[["Arayan", "Recency", "Frequency", "Monetary", "segment"]]
RFM.index = rfm_table.index.astype(int)

In [28]:
RFM.head()

,Arayan,Recency,Frequency,Monetary,segment
0,00002c75d108b05f7b43c639613208215b98ba5513b8d1...,4,1,8.600000e-08,low_interaction
1,00061a5638d32c4c876dc5117c268692dd323100f8eac4...,5,1,7.000000e-08,low_interaction
2,000756761dcdf9c2387bc25a975257346bd50ce9ef89da...,4,1,1.240000e-07,low_interaction
3,001146f2169973cd846850a5b60bd5c1e8b3dd4b78cbf2...,5,2,1.640000e-07,escalating_concerns
4,002290adbcbf3e887282dba3f0b7a543e5481af1ce5ace...,0,1,1.470000e-07,initial_contacts


In [29]:
RFM.to_csv("RFM.csv")